In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os
from io import StringIO
from tqdm import tqdm
import time
import warnings

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException
from concurrent.futures import ThreadPoolExecutor, as_completed
from multiprocessing import Process, current_process, Pool
import gc

warnings.filterwarnings("ignore")

In [2]:
# configure data path
current_path = os.path.dirname(os.path.abspath("__file__"))
data_path = os.path.join(current_path, "..", "data")

data_path = os.path.normpath(data_path)

In [3]:
def scrape_pages(loc, link):
    download_loc = os.path.join(data_path, loc)

    # check if location exists
    if not os.path.exists(download_loc):
        # if the directory does not exist, create it
        os.makedirs(download_loc)

    # configure selenium and download path
    options = Options()
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", download_loc)

    # get the state page with requests
    html_content = requests.get(link)

    # beautify the page
    soup = BeautifulSoup(html_content.text, "html.parser")

    # get tables in the hrml page with links to state broadband data
    tables = soup.find_all("table")[1]
    links = tables.find_all("a")

    # loop through the links and download the files using selenium
    for link in links:
        href = link.get("href")

        download_filename = href.split("/")[-1]
        download_path = os.path.join(download_loc, download_filename)
        temp_download_path = download_path + ".part"

        # check if download files exist
        if os.path.isfile(download_path):
            print(f"{download_filename} exist")
        else:
            # selenium driver
            driver = webdriver.Firefox(options=options)
            driver.set_page_load_timeout(5)
            try:
                driver.get(href)
            except TimeoutException:
                print("Page load timed out but continuing execution.")

            # wait for the download to complete, checking every second
            while True:
                files = os.listdir(download_loc)
                # Check if the .part file for the current download exists
                if any(
                    file.startswith(download_filename.split(".")[0])
                    and file.endswith(".part")
                    for file in files
                ):
                    print(f"Download in progress for {download_filename}...")
                    time.sleep(1)
                else:
                    # If there's no .part file, the download is complete
                    print(f"Download completed for {download_filename}.")
                    break

            driver.quit()

In [4]:
# using multiprocessor
def scrape_wrapper(args):
    return scrape_pages(*args)


# List of arguments for each dataset
datasets = [
    (
        "2017",
        "https://www.fcc.gov/form-477-broadband-deployment-data-december-2017-version-3",
    ),
    (
        "2018",
        "https://www.fcc.gov/form-477-broadband-deployment-data-december-2018-version-3",
    ),
    # ("2020", "https://www.fcc.gov/form-477-broadband-deployment-data-december-2020"),
]

# Using Pool to create a pool of worker processes
with Pool(processes=len(datasets)) as pool:
    # map the datasets to the scrape_wrapper function
    pool.map(scrape_wrapper, datasets)

AK-Fixed-Dec2018.zip exist
AL-Fixed-Dec2018.zip exist
AR-Fixed-Dec2018.zip exist
AS-Fixed-Dec2018.zip exist
AZ-Fixed-Dec2018.zip exist
CA-Fixed-Dec2018.zip exist
CO-Fixed-Dec2018.zip exist
CT-Fixed-Dec2018.zip exist
DC-Fixed-Dec2018.zip exist
DE-Fixed-Dec2018.zip exist
FL-Fixed-Dec2018.zip exist
GA-Fixed-Dec2018.zip exist
GU-Fixed-Dec2018.zip exist
HI-Fixed-Dec2018.zip exist
IA-Fixed-Dec2018.zip exist
AK-Fixed-Dec2017.zip exist
ID-Fixed-Dec2018.zip existAL-Fixed-Dec2017.zip exist

IL-Fixed-Dec2018.zip exist
IN-Fixed-Dec2018.zip existAR-Fixed-Dec2017.zip exist
KS-Fixed-Dec2018.zip exist

AS-Fixed-Dec2017.zip exist
KY-Fixed-Dec2018.zip existAZ-Fixed-Dec2017.zip exist

CA-Fixed-Dec2017.zip existLA-Fixed-Dec2018.zip exist

CO-Fixed-Dec2017.zip exist
MA-Fixed-Dec2018.zip existCT-Fixed-Dec2017.zip exist

DC-Fixed-Dec2017.zip existMD-Fixed-Dec2018.zip exist
DE-Fixed-Dec2017.zip exist

FL-Fixed-Dec2017.zip existME-Fixed-Dec2018.zip exist

GA-Fixed-Dec2017.zip existMI-Fixed-Dec2018.zip exist

G

### Download 2020 and 2010 census block relationship files

In [5]:
# download the 2020 2010 block codes concordances
# get the state page with requests
html_content = requests.get(
    "https://www.census.gov/geographies/reference-files/time-series/geo/relationship-files.2020.html#blkgrp"
)

# beautify the page
soup = BeautifulSoup(html_content.text, "html.parser")

# # get tables in the hrml page with links to state broadband data
# tables = soup.find_all('table')[1]
# links = tables.find_all('a')

In [6]:
# Find the 'ul' with a specific id
div = soup.find("div", {"id": "data-uscb-state-list-selector"})

# # Extract all 'a' tags within the 'ul'
links = div.find_all("a") if div else []

# Extract the href attribute from each link
hrefs = [link.get("href") for link in links]


def download_file(href):
    download_loc = os.path.join(data_path, "Blocks20To10")

    # check if location exists
    if not os.path.exists(download_loc):
        # if the directory does not exist, create it
        os.makedirs(download_loc)

    # configure selenium and download path
    options = Options()
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", download_loc)

    download_filename = href.split("/")[-1]
    download_path = os.path.join(download_loc, download_filename)
    temp_download_path = download_path + ".part"

    # Check if the download file already exists
    if os.path.isfile(download_path):
        print(f"{download_filename} exists")
        return

    # Start the Selenium driver
    driver = webdriver.Firefox(options=options)
    driver.set_page_load_timeout(5)

    try:
        driver.get(href)
    except TimeoutException:
        print("Page load timed out but continuing execution.")

    # Wait for the download to complete, checking every second
    while True:
        files = os.listdir(download_loc)
        # Check if the .part file for the current download exists
        if any(
            file.startswith(download_filename.split(".")[0]) and file.endswith(".part")
            for file in files
        ):
            print(f"Download in progress for {download_filename}...")
            time.sleep(1)
        else:
            # If there's no .part file, the download is complete
            print(f"Download completed for {download_filename}.")
            break

    driver.quit()


# Use ThreadPoolExecutor to download files simultaneously
with ThreadPoolExecutor(
    max_workers=8
) as executor:  # Adjust the number of workers as needed
    executor.map(download_file, hrefs)

TAB2010_TAB2020_ST04.zip exists
TAB2010_TAB2020_ST02.zip exists
TAB2010_TAB2020_ST01.zip exists
TAB2010_TAB2020_ST05.zip exists
TAB2010_TAB2020_ST08.zip exists
TAB2010_TAB2020_ST06.zip exists
TAB2010_TAB2020_ST11.zip exists
TAB2010_TAB2020_ST10.zip exists
TAB2010_TAB2020_ST09.zip exists
TAB2010_TAB2020_ST12.zip exists
TAB2010_TAB2020_ST15.zip exists
TAB2010_TAB2020_ST13.zip exists
TAB2010_TAB2020_ST16.zip exists
TAB2010_TAB2020_ST21.zip exists
TAB2010_TAB2020_ST17.zip exists
TAB2010_TAB2020_ST18.zip exists
TAB2010_TAB2020_ST22.zip exists
TAB2010_TAB2020_ST20.zip exists
TAB2010_TAB2020_ST23.zip exists
TAB2010_TAB2020_ST26.zip exists
TAB2010_TAB2020_ST25.zip exists
TAB2010_TAB2020_ST28.zip exists
TAB2010_TAB2020_ST29.zip exists
TAB2010_TAB2020_ST30.zip exists
TAB2010_TAB2020_ST33.zip exists
TAB2010_TAB2020_ST24.zip exists
TAB2010_TAB2020_ST27.zip exists
TAB2010_TAB2020_ST36.zip exists
TAB2010_TAB2020_ST37.zip exists
TAB2010_TAB2020_ST31.zip exists
TAB2010_TAB2020_ST32.zip exists
TAB2010_

### Map the FCC 2010 Blockcodes to 2020 Block Codes

In [7]:
# break the 2020, 2021, and 2019 large fcc deployment datasets
chunk_size = 200000

# latest technologies
latest_tech_codes = [43, 50]

def break_csv(year, file_name):
    csv_path = os.path.join(data_path, file_name)
    file_path = os.path.join(data_path, year)

    if not os.path.exists(file_path):
        os.makedirs(file_path)

    # use an iterator to read in chunks
    for chunk in tqdm(
        pd.read_csv(
            os.path.join(data_path, csv_path),
            chunksize=chunk_size,
            encoding="utf8",
            encoding_errors="ignore",
        )
    ):
        # Process each chunk
        for state_abbr, group_df in chunk.groupby("StateAbbr"):
            # Define the filename for each state's CSV
            filename = f"{state_abbr}_{year}.csv"

            # latest tech
            group_df["IsLatestTech"] = group_df["TechCode"].isin(latest_tech_codes)

            # consumer dataframes
            consumer_df = group_df[group_df["Consumer"] == 1]

            if consumer_df.shape[0] == 0:
                continue
    
            # Append data to the CSV if it already exists, else create a new one
            with open(os.path.join(file_path, filename), "a") as f:
                consumer_df.to_csv(f, index=False, header=f.tell() == 0)
                
def multiprocessor_wrapper(args):
    return break_csv(*args)


data = [
    ("2019", "fbd_us_without_satellite_dec2019_v1.csv"),
    ("2020", "fbd_us_without_satellite_dec2020_v1.csv"),
    ("2021", "fbd_us_without_satellite_dec2021_v1.csv"),
]

with Pool(processes=len(data)) as pool:
    # map the datasets to the scrape_wrapper function
    pool.map(multiprocessor_wrapper, data)

In [8]:
# read the files
from zipfile import ZipFile
from io import BytesIO
import json

In [9]:
# fips to state abbr relationship file
fips_st_df = pd.read_csv(
    os.path.join(data_path, "fips_states", "us-state-ansi-fips.csv")
)
fips_st_df.columns = [col.strip() for col in fips_st_df.columns]
fips_st_df.st = fips_st_df.st.apply(lambda x: str(x).zfill(2))
fips_st_df.stusps = fips_st_df.stusps.str.strip()

# create a mapping dictionary
fips_st_dict = dict(
    zip(
        fips_st_df.stusps,
        fips_st_df.st,
    )
)
# fips_st_dict

In [10]:
def zipfile_reader(zip_path):
    with ZipFile(zip_path, "r") as zip_ref:
        # extract the CSV file name from the zip file
        contained_file = zip_ref.namelist()[0]

        # determine the delimiter based on the file extension
        _, ext = os.path.splitext(contained_file)
        delimiter = "|" if ext.lower() == ".txt" else ","

        # open the file within the zip
        with zip_ref.open(contained_file) as csvfile:
            # read the file into pandas with the appropriate delimiter
            df = pd.read_csv(csvfile, delimiter=delimiter)
    return df


def process_18_17(fcc_path, rlnshp_path, year, is_partially_processed=False):
    # year in 2019, 2020, 2021
    if is_partially_processed:
        df = pd.read_csv(fcc_path)

    else:
        df = zipfile_reader(fcc_path)
        
        # consumer dataframes
        df = df[df["Consumer"] == 1]
        
    if year == "2021":
        df.rename(columns={"BlockCode": "BLKCODE2020"}, inplace=True)
        relationship_dict = {}
    else:
        # Get the relationship df of the 2010 and 2020 blocks
        df_2 = zipfile_reader(rlnshp_path)

        # concatenation using vectorized string operations
        df_2["BLKCODE2010"] = (
            df_2.STATE_2010.astype(str).str.zfill(2)
            + df_2.COUNTY_2010.astype(str).str.zfill(3)
            + df_2.TRACT_2010.astype(str).str.zfill(6)
            + df_2.BLK_2010.astype(str).str.zfill(4)
        )

        df_2["BLKCODE2020"] = (
            df_2.STATE_2020.astype(str).str.zfill(2)
            + df_2.COUNTY_2020.astype(str).str.zfill(3)
            + df_2.TRACT_2020.astype(str).str.zfill(6)
            + df_2.BLK_2020.astype(str).str.zfill(4)
        )

        df_2[["BLKCODE2020", "BLKCODE2010"]] = df_2[["BLKCODE2020", "BLKCODE2010"]].astype(
            int
        )

        # Create the relationship dictionary
        relationship_dict = dict(zip(df_2.BLKCODE2010, df_2.BLKCODE2020))

        # Filter the DataFrame to include only rows where BlockCode is a key in relationship_dict
        df = df[df.BlockCode.isin(relationship_dict)]

        # Map BlockCode to BLKCODE2020 using the relationship_dict
        df["BLKCODE2020"] = df.BlockCode.map(relationship_dict)
    
    # latest tech
    df["IsLatestTech"] = df["TechCode"].isin(latest_tech_codes)

    # group by 'BlockCode' and calculate the median 'MaxAdDown' and 'MaxAdUp'
    block_stats = (
    df.groupby("BLKCODE2020")[["MaxAdDown", "MaxAdUp"]]
    .agg(["mean", "median"])
    .reset_index()
    )

    # Renaming the columns
    block_stats.columns = ['BLKCODE2020', 'MaxAdDownMean'+ year, 'MaxAdDownMedian'+ year, 'MaxAdUpMean'+ year, 'MaxAdUpMedian'+ year]

    # calculate tech ratio
    tech_ratio = (
        df.groupby("BLKCODE2020")["IsLatestTech"]
        .mean()
        .reset_index(name="LTRatio")
    )

    df = block_stats.merge(tech_ratio, on="BLKCODE2020")

    df.rename(
        columns={
            "LTRatio": "LTRatio" + year,
            "BLKCODE2020": "GEOID20"
        },
        inplace=True,
    )

    return df, relationship_dict

In [11]:
# use the multiprocessing and threading to speed file executions
def process_files_for_year(year, batch_size=2):
    files = os.listdir(os.path.join(data_path, year))

    partially_proc_year = ["2019", "2020", "2021"]

    def process_file(file):
        if year in partially_proc_year:
            is_partially_processed = True
            state_abbr = file.split("_")[0]
            file_ending = file.endswith(".csv")
        else:
            is_partially_processed = False
            state_abbr = file.split("-")[0]
            file_ending = file.endswith(".zip")

        if state_abbr in list(fips_st_dict.keys()) and file_ending:
            # Path to state fcc deployment data
            fcc_path = os.path.join(data_path, year, file)
            file_fips = fips_st_dict[state_abbr]

            # Relationship file path
            rlnshp_name = "TAB2010_TAB2020_ST" + file_fips + ".zip"
            rlnshp_path = os.path.join(data_path, "Blocks20To10", rlnshp_name)

            df, relationship_dict = process_18_17(
                fcc_path, rlnshp_path, year, is_partially_processed
            )

            # path to save the file
            save_path = os.path.join(data_path, year + "_processed")

            if not os.path.exists(save_path):
                os.makedirs(save_path)

            output_path = os.path.join(save_path, state_abbr + "_" + year + ".csv")

            if not os.path.exists(output_path):
                # save the file for future use
                df.to_csv(output_path)

    # process files in batches
    for i in range(0, len(files), batch_size):
        batch_files = files[i : i + batch_size]
        print(batch_size)
        with ThreadPoolExecutor(max_workers=batch_size) as executor:
            future_to_file = {
                executor.submit(process_file, file): file for file in batch_files
            }
            for future in as_completed(future_to_file):
                file = future_to_file[future]
                try:
                    data = future.result()
                except Exception as exc:
                    print(f"{file} generated an exception: {exc}")
                else:
                    print(type(data))
                    # print(f'{file} is {len(data)} bytes')
                # Explicitly handle garbage collection
                gc.collect()

In [ ]:
years = ["2017", "2018", "2019", "2020", "2021"]
processes = []
max_processes = len(years)  # adjust as needed

for year in years:
    if len(processes) >= max_processes:
        for proc in processes:
            proc.join()
            processes.remove(proc)
            gc.collect()  # collect garbage to free memory
    process = Process(target=process_files_for_year, args=(year,))
    processes.append(process)
    process.start()

# wait for all processes to finish
for process in processes:
    process.join()

### Download census track shapefiles

In [13]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service

In [14]:
def download_census_tract(val, download_loc, ct_page, fips_id, PAGE_LOAD_TIMEOUT=20):
    starts_with = "tl_2020_"
    filename = "tl_2020_" + val + "_tract.zip" if fips_id == "fips_35" else "tl_2020_" + val + "_bg.zip" if fips_id == "fips_34" else "tl_2020_" + val + "_tabblock20.zip"

    if filename in os.listdir(download_loc):
        print(f"File {filename} already exists. Skipping download.")
        return

    options = webdriver.FirefoxOptions()
    options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.download.dir", download_loc)

    local_driver = webdriver.Firefox(options=options)
    local_driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)

    try:
        local_driver.get(ct_page)
        select_element = Select(local_driver.find_element(By.ID, fips_id))
        select_element.select_by_value(val)

        # Modified to use XPath for button selection
        xpath_expression = "//input[@type='button' and @onclick=\"javascript:goDownloadState('2020', 'TABBLOCK20', 'tabblock20', '33', '2020');\"]"
        download_button = local_driver.find_element(By.XPATH, xpath_expression)
        download_button.click()

        # Wait for the download to start and finish
        time.sleep(3)  # initial delay for the download to start
        while any(file.startswith(starts_with + val) and file.endswith(".part") for file in os.listdir(download_loc)):
            time.sleep(1)

    except Exception as e:
        print(f"An error occurred with value {val}: {e}")
    
    finally:
        local_driver.quit()

def download_census_files(loc, ct_page, fips_id):
    download_loc = os.path.join(data_path, loc)
    PAGE_LOAD_TIMEOUT = 20

    if not os.path.exists(download_loc):
        os.makedirs(download_loc)

    driver = webdriver.Firefox()
    driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)
    driver.get(ct_page)

    try:
        select_element = Select(driver.find_element(By.ID, fips_id))
        values = [option.get_attribute("value") for option in select_element.options if option.get_attribute("value")]

        driver.quit()

        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(download_census_tract, val, download_loc, ct_page, fips_id, PAGE_LOAD_TIMEOUT) for val in values]

            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    print(f"Exception occurred: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")
        driver.quit()

In [15]:
loc = "CensusTractShp"
ct_page = "https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2020&layergroup=Census+Tracts"
fips_id = "fips_35"

# download census tract shapefiles
# download_census_files(loc, ct_page, fips_id)

In [ ]:
loc = "CensusBlockShp"
ct_page = "https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2020&layergroup=Blocks+%282020%29"
fips_id = "fips_33"

# download census tract shapefiles
download_census_files(loc, ct_page, fips_id)

In [30]:
loc = "CensusBlockGroups"
ct_page = "https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2020&layergroup=Block+Groups"
fips_id = "fips_34"

# download census tract shapefiles
download_census_files(loc, ct_page, fips_id)

File tl_2020_01_bg.zip already exists. Skipping download.
File tl_2020_02_bg.zip already exists. Skipping download.
File tl_2020_04_bg.zip already exists. Skipping download.
File tl_2020_60_bg.zip already exists. Skipping download.
File tl_2020_05_bg.zip already exists. Skipping download.
File tl_2020_06_bg.zip already exists. Skipping download.
File tl_2020_08_bg.zip already exists. Skipping download.
File tl_2020_09_bg.zip already exists. Skipping download.
File tl_2020_69_bg.zip already exists. Skipping download.
File tl_2020_10_bg.zip already exists. Skipping download.
File tl_2020_12_bg.zip already exists. Skipping download.
File tl_2020_11_bg.zip already exists. Skipping download.
File tl_2020_13_bg.zip already exists. Skipping download.
File tl_2020_66_bg.zip already exists. Skipping download.
File tl_2020_16_bg.zip already exists. Skipping download.
File tl_2020_15_bg.zip already exists. Skipping download.
File tl_2020_18_bg.zip already exists. Skipping download.
File tl_2020_1

### Clean other demographic factors

In [24]:
# clean income data, esp with uncertainties such as 25000-, 250000+ and - (nans)
def transform_income(value):
    if type(value) == str:
        # Remove commas
        value = value.replace(",", "")

        # Handle different cases
        if "-" in value:
            if value.endswith("-") and len(value) != 1:
                # One-sided range, use the provided value
                return float(value.replace("-", ""))
            elif value.startswith("-"):
                # Missing or uncertain data
                return None  # Or use 0, or a specific strategy for missing data
            else:
                # Range, take the average
                low, high = value.split("-")
                return (float(low) + float(high)) / 2
        elif "+" in value:
            # Open-ended value, use the provided number
            return float(value.replace("+", ""))
        elif value == "-":
            # Missing or uncertain data
            return None  # Or use mean/median of the column
        else:
            # Regular value
            return float(value)
    return value

In [29]:
income_data_path = os.path.join(
    data_path, "median_income", "ACSDT5Y2021.B19049-Data.csv"
)
edu_data_path = os.path.join(data_path, "education", "ACSDT5Y2021.B15003-Data.csv")
urban_rural_data_path = os.path.join(
    data_path, "urban_rural", "DECENNIALDHC2020.P2-Data.csv"
)
internet_avl_data_path = os.path.join(
    data_path, "internet_availability", "ACSDT5Y2021.B28002-Data.csv"
)

# read the csv files
# income data
income_df = pd.read_csv(income_data_path, usecols=["GEO_ID", "NAME", "B19049_001E"])
income_df.rename(columns={"B19049_001E": "median_income"}, inplace=True)
income_df.drop(0, inplace=True)
income_df["median_income"] = income_df["median_income"].apply(transform_income)
income_df["median_income"].fillna(income_df["median_income"].mean(), inplace=True)

# education data
edu_df = pd.read_csv(
    edu_data_path, usecols=["GEO_ID", "NAME", "B15003_001E", "B15003_022E"]
)
edu_df.drop(0, inplace=True)
edu_df["with_degree"] = edu_df["B15003_022E"].astype(int) / edu_df[
    "B15003_001E"
].astype(int)

# ratio of rural to urban
urban_df = pd.read_csv(
    urban_rural_data_path, usecols=["GEO_ID", "NAME", "P2_001N", "P2_002N"]
)
urban_df.drop(0, inplace=True)
urban_df["urban_pop"] = urban_df["P2_002N"].astype(int) / urban_df["P2_001N"].astype(
    int
)
urban_df.rename(columns={"P2_001N": "POP20"}, inplace=True)

# with broadband of any type
int_avl_df = pd.read_csv(
    internet_avl_data_path, usecols=["GEO_ID", "NAME", "B28002_001E", "B28002_004E"]
)
int_avl_df.drop(0, inplace=True)
int_avl_df["with_broadband"] = int_avl_df["B28002_004E"].astype(int) / int_avl_df[
    "B28002_001E"
].astype(int)

# merge the datasets and extract the columns of interest

data = (
    income_df[["GEO_ID", "median_income"]]
    .merge(edu_df[["GEO_ID", "with_degree"]], on="GEO_ID")
    .merge(urban_df[["GEO_ID", "urban_pop", "POP20"]], on="GEO_ID")
    .merge(int_avl_df[["GEO_ID", "with_broadband"]], on="GEO_ID")
)

output_path = os.path.join(data_path, "in_urb_ed_brb.csv")

# get census track
data["GEOID"] = data.GEO_ID.str[9:]
data.to_csv(output_path, index=False)